# PDS: Assignment 3
## For 10 points (not 5 points as usual)

### Deadline: June 1 (until 23:59)


**Instructions:** All answers should be filled in the notebook and then submitted to Moodle. For theoretical question you can use markdown and LaTeX. Name of notebook should be in following format:
 - Name_group_Assignment_3.ipynb (ex. Elon_Musk_1929_Assignment_3.ipynb)
 - Try to use fewer cells for compilation (for example, you can use print for several answers, instead of printing each answer on a separate cell)
 - **PLEASE:** submit only one jupyter notebook (no zip or csv files) and with your names as in the instruction. Ohterwise, there will be punishment for 2 points.

## Q1 (30%)
Use Credit.csv dataset. Do the following tasks:
1. Split your dataset to train and test sets (20% for testing) with random state equal to 2021. Imagine that you can use test set only for evaluation at the end.
2. Using any model of your choice, predict Balance. To evaluate performance of the trained model, obtain MSE of training set, 5-fold CV, 10-fold CV, and test set. Present all results in the form of tables (dataframes).
3. Instead of using CV, using any random split obtain validation set of the same size as the test set. Train the model on training part, and evaluate performance of the trained model on training, validation and test sets. Present all results in the form of tables (dataframes).
4. Now, repeat the step above 5 times with 5 different random splits. You can save results in five different plots. 
5. Comment the results obtained using CV and validation sets.


*Note: You are allowed to use sklearn, but not statsmodels.*

*Hint: To get validation set, you can use train_test_split function two times (if you use it properly).*

In [73]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold, GridSearchCV,cross_val_score

In [2]:
df_credit = pd.read_csv('Credit.csv')
df_credit.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [98]:
#Task 1
cols = df_credit[['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education']]

du = pd.get_dummies(df_credit[['Gender', 'Student', 'Married', 'Ethnicity']])

scaler = MinMaxScaler()
cols[['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education']] = scaler.fit_transform(numeric)

X = pd.concat([cols, du], axis = 1)
y = df_credit['Balance']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)

<ipython-input-98-a876e7a9befe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cols[['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education']] = scaler.fit_transform(numeric)
C:\Users\Damir\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [103]:
#Task 2
lr = LinearRegression().fit(X_train,y_train)

train = metrics.mean_squared_error(y_train, lr.predict(X_train))
test = metrics.mean_squared_error(y_test, lr.predict(X_test))

train_5 = abs(cross_val_score(lr, X_train, y_train, scoring='neg_mean_squared_error', cv=5))
test_5 = abs(cross_val_score(lr, X_test, y_test, scoring='neg_mean_squared_error', cv=5))

train_10 = abs(cross_val_score(lr, X_train, y_train, scoring='neg_mean_squared_error', cv=10))
test_10 = abs(cross_val_score(lr, X_test, y_test, scoring='neg_mean_squared_error', cv=10))

li = [['Mean Square Error on train set', train],
      ['Mean Square Error on test set', test],
      ['5 fold CV on train set', train_5.mean()],
      ['10 fold CV on train set', train_10.mean()],
      ['5 fold CV on test set', test_5.mean()],
      ['10 fold CV on test set', test_10.mean()]]

new_df = pd.DataFrame(li, columns=['Set','MSE'])
new_df

,Set,MSE
0,Mean Square Error on train set,772.692883
1,Mean Square Error on test set,777.321383
2,5 fold CV on train set,850.489674
3,10 fold CV on train set,827.086652
4,5 fold CV on test set,896.559983
5,10 fold CV on test set,862.520154


In [104]:
#Task 3
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state=2021)

lr = LinearRegression().fit(X_train, y_train)

train = metrics.mean_squared_error(y_train, lnr.predict(X_train))
valid = metrics.mean_squared_error(y_val, lnr.predict(X_val))
test = metrics.mean_squared_error(y_test, lnr.predict(X_test))


new_df = pd.DataFrame({'Set':['Train','Validation','Test'], 'MSE':[train, valid, test]})
new_df

,Set,MSE
0,Train,799.341375
1,Validation,842.178917
2,Test,879.471744


In [106]:
#Task 4
train_set = []
valid_set = []
test_set = []

li = [0.1, 0.2, 0.3, 0.4, 0.5]

y = df_credit['Balance']


for rand in li:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= rand, random_state=2021)
    
    lr_1 = LinearRegression().fit(X_train,y_train)
    
    train = mean_squared_error(y_train, lr_1.predict(X_train))
    valid = mean_squared_error(y_val, lr_1.predict(X_val))
    test = mean_squared_error(y_test, lr_1.predict(X_test))
    
    train_set.append(train)
    valid_set.append(valid)
    test_set.append(test)

In [107]:
new_df = pd.DataFrame({'Set ':['Train','Validation','Test'], '0.1 validation':[trainSet[0],validSet[0],testSet[0]], 
                     '0.2 validation':[trainSet[1], validSet[1],testSet[1]],
                     '0.3 validation':[trainSet[2], validSet[2],testSet[2]],
                     '0.4 validation':[trainSet[3], validSet[3],testSet[3]],
                     '0.5 validation':[trainSet[4], validSet[4],testSet[4]]})
new_df.head()

,Set,0.1 validation,0.2 validation,0.3 validation,0.4 validation,0.5 validation
0,Train,785.185007,799.355203,787.793969,802.936518,828.608951
1,Validation,694.059975,709.405261,802.388282,821.511306,831.476915
2,Test,780.719130,805.518079,844.333219,872.192774,897.241681


We can observe from the results, when we use validation set the results of train and test sets are more compared to results when CV was used.

## Q2 (20%)
Use boston.csv dataset. Do the following tasks: 
1. Split your dataset to train and test sets (20% for testing) with random state equal to 2021. Imagine that you can use test set only for evaluation at the end.

2. Your task is to predict target variable - **medv**.

3. Use Ridge regression with alpha (penalty term) varying from 0 to some large value of alpha, at which all coefficients take value of 0. Using Cross-Validation select the most optimal value of alpha. 

4. For the selected value of alpha train your model on the training set (80%). And then evaluate on test set (20%). 

5. Show the coefficients of the final model.

*Note: You should use regular Ridge model from sklearn (not RidgeCV) and apply CV on it.*

In [66]:
df_boston = pd.read_csv('boston.csv')
df_boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [63]:
#Task 1
X = df_boston.iloc[:,:-1].values
y = df_boston.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)

In [10]:
#Task 3
model = Ridge()
cross_validation = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)

gridSearch = GridSearchCV(model, grid, scoring = 'neg_root_mean_squared_error', cv = cross_validation)
res = gridSearch.fit(X_train, y_train)

print('Alpha: %s' % res.best_params_)

In [12]:
#Task 4
rr_model = Ridge(alpha = 0.07, normalize = True)
rr_model.fit(X_train, y_train) 

pred_testRidge= rr_model.predict(X_test)

print('MSE for test data:', mean_squared_error(y_test, pred_testRidge)) 

MSE for test data: 22.07320004494829


In [70]:
X = df_boston.drop('medv', axis = 1)
y = df_boston['medv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)

In [71]:
#Task 5
print(pd.Series(rr_model.coef_, index = X_train.columns))

crim       -0.094329
zn          0.038655
indus      -0.052146
chas        3.705767
nox       -12.401701
rm          4.126202
age         0.004338
dis        -1.181513
rad         0.189612
tax        -0.005581
ptratio    -0.786476
black       0.009430
lstat      -0.514113
dtype: float64


## Q3 (20%)
Again use boston.csv dataset. Do the following tasks: 
1. Split your dataset to train and test sets (20% for testing) with random state equal to 2021. Imagine that you can use test set only for evaluation at the end.

2. Again, your task is to predict target variable - **medv**.

3. Use Lasso regression with alpha (penalty term) varying from 0 to some large value of alpha, at which all coefficients take value of 0. Using Cross-Validation select the most optimal value of alpha. 

4. For the selected value of alpha train your model on the training set (80%). And then evaluate on test set (20%). 

5. Show the coefficients of the final model.

6. Compare your results (i.e. test MSE) and coefficients with the Ridge Model.

*Note: In this case you should use LassoCV (not Lasso and apply CV separately).*

In [92]:
df_boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [93]:
#Task 3
modelLasso = LassoCV(alphas = np.arange(0, 1, 0.01), cv = cross_validation)
modelLasso.fit(X_train, y_train)

print('Alpha: %f' % modelLasso.alpha_)

C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4072.815777529275, tolerance: 3.2602504517906334
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4145.122805331772, 

C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4063.0368313482686, tolerance: 3.1098815467032965
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3862.3475517958977, tolerance: 3.366248777472528
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: UserWarning: Coordinate descent with alpha=0 may lead to unexpect

Alpha: 0.000000


C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3958.9353662732537, tolerance: 3.249628296703297
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1336: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X, y)
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Damir\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4414.208820885549, 

In [94]:
#Train 4
pred_testLasso= modelLasso.predict(X_test)

print('MSE for test data:', mean_squared_error(y_test,pred_testLasso))

MSE for test data: 23.299928845559098


In [97]:
#Train 5

print(pd.Series(modelLasso.coef_, index=X_train.columns)) 

crim       -0.114557
zn          0.053234
indus       0.003283
chas        3.508465
nox       -18.135667
rm          3.825239
age         0.011058
dis        -1.529967
rad         0.339221
tax        -0.011868
ptratio    -0.884215
black       0.009528
lstat      -0.578169
dtype: float64


In this case, the Ridge Model shows a quite better results for this dataset 22.07 (MSE), while MSE for Lasso test data is 23.29

## Q4 (30%)
Use Wage.csv dataset. Do the following tasks:
1. Split your dataset to train and test sets (20% for testing) with random state equal to 2021. Imagine that you can use test set only for evaluation at the end.
2. Perform some data preprocessing.
3. Build Tree model to predict wage value. Set up parameters as you want or use default parameters. Evaluate the performance on test set.
4. Do the same but with Bagging.
5. Do the same but with Random Forest. 
6. Compare their resuts. Which model it is better to use. 
7. Now, for Random Forest model select the parameter that should be tuned and use CV to tune this parameter.
8. After selecting the parameter, run the model again. Show it's performance on test set. 

In [15]:
#Task 2
df_wage = pd.read_csv('Wage.csv')
df_wage = df_wage.drop('region', axis = 1)
df_wage = df_wage.drop('sex', axis = 1)
df_wage = df_wage.drop('Unnamed: 0', axis =1)
le = preprocessing.LabelEncoder()
df_wage =df_wage.apply(le.fit_transform)

df_wage.head()

,year,age,maritl,race,education,jobclass,health,health_ins,logwage,wage
0,3,0,0,0,0,0,0,1,125,125
1,1,6,0,0,3,1,1,1,104,104
2,0,27,1,0,2,0,0,0,353,353
3,0,25,1,2,3,1,1,0,425,425
4,2,32,3,0,1,1,0,0,125,125


In [16]:
#Task 1
cols = ['year','age','maritl','race','education',
             'jobclass','health','health_ins', 'logwage']

X = df_wage[cols]
y = df_wage['wage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2021)

In [76]:
#Task 3
tree = DecisionTreeRegressor()
tree = tree.fit(X_train,y_train)

y_pred = tree.predict(X_test)

print("MSE:", metrics.mean_squared_error(y_test, y_pred))

MSE: 22.943039215686277


In [77]:
#Task 4
bag = BaggingRegressor()
bag = bag.fit(X_train, y_train)

y_pred = bag.predict(X_test)

print("MSE:", metrics.mean_squared_error(y_test, y_pred))

MSE: 13.022024509803922


In [78]:
#Task 5
forest = RandomForestRegressor()
forest = forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

print("MSE:", metrics.mean_squared_error(y_test, y_pred))

MSE: 13.079944313725472


We can see that RandomForest and Bagging are showing approximately the same results, whereas DecisionTree is the worst one.

In [88]:
#Task 7
tree_params = {
    "max_depth": [10,15],
    "max_features": [2, 3],
    'n_estimators': [100, 200, 300],
}

tree_grid = GridSearchCV(forest, tree_params, cv=5, n_jobs=-1, verbose=True)

tree_grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    5.4s finished


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=15, max_features=3,
                                             min_samples_leaf=3,
                                             min_samples_split=5,
                                             n_estimators=300),
             n_jobs=-1,
             param_grid={'max_depth': [10, 15], 'max_features': [2, 3],
                         'n_estimators': [100, 200, 300]},
             verbose=True)

In [89]:
tree_grid.best_params_

{'max_depth': 10, 'max_features': 3, 'n_estimators': 300}

In [91]:
#Task 8
forest = RandomForestRegressor(n_estimators = 300, max_depth = 10, max_features = 3)
forest = forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

print("MSE:", metrics.mean_squared_error(y_test, y_pred))

MSE: 8.685776712430632


From the results above we can see that tuned RandomForest shows better results than other methods.

---

### Evaluation

| Question | Mark     | Comment   
|:-------:|:--------:|:----------------------
| 1       |   30/30    |     
| 2       |   20/20   | 
| 3       |   20/20   |     
| 4       |   30/30   | 
|**Total**|**100/100**  | 
